In [1]:
!pip install google-search-results

## 環境変数読み込み
- OPENAI_API_KEY: OPENAIのAPIを使うために必要
- SERPAPI_API_KEY: Serp APIを使うために必要

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### Agentで使うツールを指定

In [3]:
from langchain import SerpAPIWrapper, LLMMathChain
from langchain.llms import OpenAI
from langchain.agents import load_tools, Tool, initialize_agent

llm = OpenAI(temperature=0.7)

tool_names = ["serpapi", "llm-math"]

tools = load_tools(tool_names, llm=llm)


agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

### Agentの実行

In [4]:
agent.run("最新のドル円の為替レートを2乗してください")



> Entering new AgentExecutor chain...
 検索してドル円の為替レートを探す
Action: Search
Action Input: 最新のドル円の為替レート
Observation: 【NQNニューヨーク=横内理恵】25日のニューヨーク外国為替市場で円相場は続落し、前日比60銭円安・ドル高の1ドル=146円40～50銭で取引を終えた。
Thought: 見つかったレートを2乗してあげる
Action: Calculator
Action Input: 146.4 * 146.4
Observation: Answer: 21432.960000000003
Thought: 答えを確認する
Final Answer: 21432.960000000003

> Finished chain.


'21432.960000000003'

### Toolの説明を手動で指定して実行

In [6]:
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name = "検索をする道具",
        func=search.run,
        description="為替の検索などができます"
    ),
    Tool(
        name="計算機",
        func=llm_math_chain.run,
        description="計算をすることができます"
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("最新のドル円の為替レートを2乗してください")



> Entering new AgentExecutor chain...
 為替レートを検索することから始めよう
Action: 検索をする道具
Action Input: 最新のドル円の為替レート
Observation: 【NQNニューヨーク=横内理恵】25日のニューヨーク外国為替市場で円相場は続落し、前日比60銭円安・ドル高の1ドル=146円40～50銭で取引を終えた。
Thought: 為替レートを計算機で2乗する
Action: 計算機
Action Input: 146.40 x 146.40

> Entering new LLMMathChain chain...
146.40 x 146.40```text
146.40**2
```
...numexpr.evaluate("146.40**2")...

Answer: 21432.960000000003
> Finished chain.

Observation: Answer: 21432.960000000003
Thought: 答えを確認した
Final Answer: 21432.960000000003

> Finished chain.


'21432.960000000003'

### 単発で動かすこともできる

In [7]:
search.run("最新の為替")

'【NQNニューヨーク=横内理恵】25日のニューヨーク外国為替市場で円相場は続落し、前日比60銭円安・ドル高の1ドル=146円40～50銭で取引を終えた。'

In [8]:
llm_math_chain.run("2*2")



> Entering new LLMMathChain chain...
2*2```text
2*2
```
...numexpr.evaluate("2*2")...

Answer: 4
> Finished chain.


'Answer: 4'

### promptの中身を確認

In [9]:
print(llm_math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [10]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

検索をする道具: 為替の検索などができます
計算機: 計算をすることができます

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [検索をする道具, 計算機]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
